<hr>

## Detection vs Recognition vs Tracking
<hr>

- 차이점
    - 검출(Detection): 영상에서 찾고자 하는 대상의 위치와 크기를 알아내는 작업
    - 인식(Recognition): 주어진 영상(객체)이 무엇인지 판별하는 작업(Classification, Identification)
    - 추적(Tracking): 동영상에서 이전 프레임에 있던 특정 대상의 현재 프레임에서의 위치 변화를 알아내는 작업
        - e.g., Mean Shift, CamShift, Optical flow, Trackers in OpenCV 3.x
    - 만약 사람의 얼굴이 포함된 영상에서 누구인지는 상관없이 `모든 얼굴을 찾는 것을 검출`, 그 중 `특정 인물을 찾는 것을 인식`,<br> 동영상이나 여러 장의 영상에서 `특정 대상의 변화를 찾는 것이 추적`이다.
- extra: Tracing vs Tracking
    - to Trace: 이미 주어진 경로(객체의 외곽선 등)의 자취를 쫒아가며 그 대상의 위치와 형태를 알아내는 작업
    - to Track: 동영상에서 특정 대상의 변화를 뒤쫒으며 추적하여 변화 과정을 알아내는 작업
    - 만약 철수가 `내 과자를 허락없이 먹은 것을 목격하여` 철수를 잡아 따지기 위해 철수의 뒤를 쫒아가는 것을 `Tracking`,<br>
    `누군가 내 과자를 허락없이 먹었는데` 주변에 과자 부스러기가 떨어져있고, 그 부스러기의 자취를 쫒아가봤더니 철수의 입가에<br>
    내 과자로 추정되는 과자 부스러기가 묻어 있는 것을 발견하여 철수에게 따지러 가는 것이 `Tracing`이다.

## <font color = "#CC3D3D">Case #1: Background Subtraction</font>

<hr>

### Step #1: Concept
<hr>

- 배경 차분(Background Subtraction: BS)
    - 등록된 배경 모델과 현재 입력 프레임과의 차영상을 이용하여 전경 객체를 검출하는 기법이다.
    - 움직이는 전경 객체 검출을 위한 기본적인 방법이다.
    - <img src="images/markdown/.png" width="600">

<hr>

### Step #2: Static Background Subtraction
<hr>

In [23]:
# 정적 배경을 이용한 전경 객체 검출 예제
import sys

import cv2

capture = cv2.VideoCapture("videos/PETS2000.avi")
# capture = cv2.VideoCapture("videos/vtest.avi")

if not capture.isOpened():
    print("Video open failed!")
    sys.exit()
else:
    print("Video open succeed!")

# 첫 번째 프레임을 배경 영상으로 등록
retval, background = capture.read()

if not retval:
    print("Background image registration failed!")
    sys.exit()
else:
    print("Background image registration succeed!")

# 컬러 값이 반드시 필요한 상황이 아니고, 연산 속도를 증가시키기 위해 그레이스케일로 변환
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
background_gray = cv2.GaussianBlur(background_gray, (0, 0), 1)

# 비디오 매 프레임 처리
while True:
    retval, frame = capture.read()
    
    if not retval:
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (0, 0), 1)
    
    # 차영상 구하기 & 이진화
    difference = cv2.absdiff(background_gray, frame_gray)
    _, difference = cv2.threshold(difference, 30, 255, cv2.THRESH_BINARY)
    
    # 레이블링 기법을 이용하여 원본 프레임에 바운딩 박스 표시
    count, _, stats, _ = cv2.connectedComponentsWithStats(difference)
    
    for i in range(1, count):
        x, y, w, h, s = stats[i]
        
        if s < 100:
            continue
        
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)
    
    # OpenCV 가상 윈도우로 출력
    cv2.imshow("Frame", frame)
    cv2.imshow("Difference", difference)

    if cv2.waitKey(30) == 27:
        print("Video was interrupted!")
        break

capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

Video open succeed!
Background image registration succeed!


-1

<hr>

### Step #3: Moving Average Background Subtraction
<hr>

- 정적 배경 모델 사용 시 문제점
    - 미리 등록된 디준 영상이 실제 배경과 크게 달라질 경우 오동작을 일으킬 수 있다.
        - e.g., 그림자 등의 영향으로 인한 `조도 변경`, 시점 변경으로 인한 `밝기 감소`, `새로운 객체가 화면에 고정`되는 경우
        - <img src="images/markdown/.png" width="600">

<hr>

- 평균 연산에 의한 배경 영상 생성
    - 움직이는 객체가 존재하는 수백 장의 입력 영상으로부터 `평균 영상`을 구한다.
    - <img src="images/markdown/.png" width="600">
    - 다만 수백 장의 이전 프레임을 버퍼에 저장하려면 `대용량 메모리가 필요`하다.

<hr>

- 이동 평균(Moving Average)
    - 수백 장의 영상을 저장하는 대신 `매 프레임이 들어올 때마다 평균 영상을 갱신`한다.
    - 평균 연산에 비해 대용량 버퍼 메모리가 필요하지 않다.
    - 내부 연산은 `cv2.addWeighted()` 함수와 같은 `가중치 합`을 구하는 형태이며, 이전 프레임까지의 `배경 영상`과 `현재 프레임`의<br>
    `가중치 합`을 계산하여 현재 프레임에서의 `배경 영상`을 업데이트 하는 방법이다.

$$B(x,y,t)=\alpha\cdot I(x,y,t)+(1-\alpha)\cdot B(x,y,t-1)$$
$$B(x,y,t)={\scriptstyle\text{갱신된 배경 영상}}$$
$$\alpha={\scriptstyle\text{현재 프레임에 대한 가중치}},\ (0<\alpha<1)$$
$$I(x,y,t)={\scriptstyle\text{현재 프레임}}$$
$$B(x,y,t-1)={\scriptstyle\text{이전 프레임까지의 배경 영상}}$$

<hr>

### Step #3-1: OpenCV function
<hr>

> `이동 평균 계산을 위한 가중치 누적 함수`

$$\mathsf{{\color{RoyalBlue}cv2.}{\color{Tan}accumulateWeighted}(src, dst, alpha, mask)\rightarrow }$$
- src: 입력 영상
- dst: 축적 영상(결과 영상)
- alpha: (입력 영상에 대한) 가중치
- mask: 마스크 연산 시 사용할 마스크 영상
- `참고사항:`
    - src: 1 or 3 channel, 8 bit or 32 bit float type
    - dst: src와 동일 채널, 32 bit or 64 bit float type
    - alpha: `1을 지정`하면 이전까지의 배경 영상을 무시하고 `현재 프레임을 배경 영상으로 사용`한다.<br>
    반대로 `0을 지정`하면 현재 프레임에 대한 정보가 사라지고 처음에 모델로 사용한 배경 영상을 계속 사용한다.`(정적 배경 모델)`
        - 일반적으로 `0에 가까운 값`을 지정한다.`(e.g., alpha = 0.01 or less)`

<hr>

### Step #3-2: Implementation example
<hr>

In [22]:
# 이동 평균에 의한 배경 차분 예제
import sys

import cv2
import numpy as np

capture = cv2.VideoCapture("videos/PETS2000.avi")
# capture = cv2.VideoCapture("videos/vtest.avi")

if not capture.isOpened():
    print("Video open failed!")
    sys.exit()
else:
    print("Video open succeed!")

# 첫 번째 프레임을 배경 영상으로 등록
retval, background = capture.read()

if not retval:
    print("Background image registration failed!")
    sys.exit()
else:
    print("Background image registration succeed!")

# background: uint8 배경, background_float: float32 배경
background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
background = cv2.GaussianBlur(background, (0, 0), 1)
background_float = background.astype(np.float32)

# 비디오 매 프레임 처리
while True:
    retval, frame = capture.read()
    
    if not retval:
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (0, 0), 1)
    
    # background_float: 이전 프레임까지의 배경(float32), background: 갱신된 배경(uint8)
    cv2.accumulateWeighted(frame_gray, background_float, 0.01)
    background = background_float.astype(np.uint8)
        
    # 차영상 구하기 & 이진화
    difference = cv2.absdiff(background, frame_gray)
    _, difference = cv2.threshold(difference, 30, 255, cv2.THRESH_BINARY)
    
    # 레이블링 기법을 이용한 원본 프레임의 움직이는 객체에 바운딩 박스 표시
    count, _, stats, _ = cv2.connectedComponentsWithStats(difference)
    
    for i in range(1, count):
        x, y, w, h, s = stats[i]
        
        if s < 100:
            continue
        
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)
    
    # OpenCV 가상 윈도우로 출력
    cv2.imshow("Frame", frame)
    cv2.imshow("Difference", difference)
    cv2.imshow("Background", background)

    if cv2.waitKey(30) == 27:
        print("Video was interrupted!")
        break

capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

Video open succeed!
Background image registration succeed!


-1

<hr>

### Step #4: MOG Background Model
<hr>

- MOG
    - Mixture of Gaussian, GMM(Gaussian Mixture Model)
    - 각 픽셀에 대해 MOG 확률 모델을 설정하여 배경과 전경을 구분하는 일종의 데이터 분석 기법이다.

<hr>

- 다양한 배경 모델 구성 방법
    - <img src="images/markdown/.png" width="600">

<hr>

- dfdfdfd
    - <img src="images/markdown/.png" width="600">

<hr>

- dfdfdf
    - <img src="images/markdown/.png" width="600">

<hr>

### Step #4-1: OpenCV function
<hr>

> `배경 차분 알고리즘 객체 생성`

$$\mathsf{{\color{RoyalBlue}cv2.}{\color{Tan}createBackgroundSubtractorMOG2}(, history, varThreshold, detectShadows)\rightarrow retval}$$
- history: 히스토리 길이
- varThreshold: variance threshold. 픽셀과 모델 사이의 마할라노비스 거리(Mahalanobis distance) 제곱에 대한 임계값
- detectShadows: 그림자 검출 여부
- retval: cv2.BackgroundSubtractor 클래스 인스턴스(객체)
- `참고사항:`
    - history: 이전(과거) 프레임을 몇 개까지 사용할지를 지정한다.
        - `기본값은 500`
    - varThreshold: 해당 픽셀이 배경 모델에 의해 잘 표현되는 지를 판단한다. 이 파라미터는 배경 영상 갱신에 영향을 주지 않는다.
        - `기본값은 16`
    - detectShadows: `기본값은 True`

$$\mathsf{{\color{RoyalBlue}cv2.}{\color{Tan}createBackgroundSubtractorKNN}(, history, dist2Threshold, detectShadows)\rightarrow }$$
- history: 
- dist2Threshold: distance to threshold. 픽셀과 샘플 사이의 거리 제곱에 대한 임계값
- detectShadows: 
- retval: cv2.BackgroundSubtractor 클래스 인스턴스(객체)
- `참고사항:`
    - history: 이전(과거) 프레임을 몇 개까지 사용할지를 지정한다.
        - `기본값은 500`
    - dist2Threshold: 해당 픽셀이 샘플에 가까운 지를 판단한다. 이 파라미터는 배경 영상 갱신에 영향을 주지 않는다.
        - `기본값은 400`
    - detectShadows: `기본값은 True`

<hr>

> `전경 객체 마스크 생성 함수`

$$\mathsf{{\color{RoyalBlue}cv2.BackgroundSubtractor.}{\color{Tan}apply}(image, fgmask, learningRate)\rightarrow fgmask}$$
- image: (입력) 다음 비디로 프레임
- fgmask: foreground mask. (출력) 전경 마스크 영상
- learningRate: 배경 모델 학습율(속도) 지정
- `참고사항:`
    - fgmask: 8 bit 이진 영상(0 or 255). `detectShadows = True`이면 `0 or 128 or 255`로 이루어진 영상을 반환한다.
    - learningRate: 0 ~ 1 사이의 실수를 지정한다. `이동 평균의 가중치(alpha)`와 같은 역할이다.
        - 0: 배경 모델을 갱신하지 않음
        - 1: 매 프레임마다 배경 모델을 새로 만듦
        - -1: 자동으로 결정됨. `기본값`

<hr>

> `배경 영상 반환 함수`

$$\mathsf{{\color{RoyalBlue}cv2.BackgroundSubtractor.}{\color{Tan}getBackgroundImage}(, backgroundImage)\rightarrow backgroundImage}$$
- backgroundImage: (출력) 학습된 배경 영상

In [ ]:
import cv2
help(cv2.BackgroundSubtractor)

In [ ]:
import cv2
help(cv2.createBackgroundSubtractorMOG2)

In [ ]:
import cv2
help(cv2.createBackgroundSubtractorKNN)

In [ ]:
import cv2
help(cv2.BackgroundSubtractor.apply)

In [ ]:
import cv2
help(cv2.BackgroundSubtractor.getBackgroundImage)

<hr>

### Step #4-2: Implementation example
<hr>

In [18]:
# MOG 기법을 이용한 배경 생성 및 전경 객체 검출 예제
import sys

import cv2
import numpy as np

capture = cv2.VideoCapture("videos/PETS2000.avi")
# capture = cv2.VideoCapture("videos/vtest.avi")

if not capture.isOpened():
    print("Video open failed!")
    sys.exit()
else:
    print("Video open succeed!")

# 배경 차분 알고리즘 객체 생성
# BS = cv2.createBackgroundSubtractorMOG2()
BS = cv2.createBackgroundSubtractorKNN()

# 비디오 매 프레임 처리
while True:
    retval, frame = capture.read()
    
    if not retval:
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    fgmask = BS.apply(frame_gray)
    background = BS.getBackgroundImage()
    
    # 레이블링 기법을 이용하여 바운딩 박스 표시
    count, _, stats, _ = cv2.connectedComponentsWithStats(fgmask)
    
    for i in range(1, count):
        x, y, w, h, s = stats[i]
        
        if s < 100:
            continue
        
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)
    
    cv2.imshow("Frame", frame)
    cv2.imshow("Difference", fgmask)
    cv2.imshow("Background", background)
    
    if cv2.waitKey(30) == 27:
        print("Video was interrupted!")
        break

capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

Video open succeed!


-1

## <font color = "#CC3D3D">Case #2: Mean Shift Algorithm</font>

<hr>

### Step #1: Concept
<hr>

- 평균 이동(Mean Shift)
    - 평균 이동이라고 하는 것은 데이터사이언스 분야에서 `어떤 데이터의 분포를 분석하는 기법 중 하나`이고, 어떤 데이터의 분포에서<br>
    `데이터가 가장 밀집되어 있는 부분`을 찾아내기 위한 기법이다.
        - e.g., 어떤 데이터가 가우시안 분포 형태를 이루고 있을 때 그 `가우시안 분포의 평균(mode)`를 찾아내는 방법이다.<br>
        경우에 따라서 `모드 검출(mode seeking) 알고리즘`이라는 용어로도 사용한다.
    - 평균 이동 알고리즘은 국지적 평균(Local Mean)을 탐색하면서 이동한다.

<hr>

- 평균 이동 알고리즘 작동 예시
    - <img src="images/markdown/MeanShifted.png" width="200">

<hr>

- 평균 이동 알고리즘의 단점
    - <img src="images/markdown/.png" width="600">

<hr>

### Step #2: OpenCV function
<hr>

> `평균 이동 알고리즘을 이용한 트래킹 함수`

$$\mathsf{{\color{RoyalBlue}cv2.}{\color{Tan}meanShift}(probImage, window, criteria)\rightarrow retval}$$
- probImage: probabillity image. 입력 확률 영상
- window: `(x, y, w, h)` 형태의 튜플
- criteria: 알고리즘 종료 기준으로 `(type, maxCount, epsilon)` 형태의 튜플
- retval: 알고리즘 내부 반복 횟수
- `참고사항:`
    - probImage: 일반적으로 관심 객체에 대한 `히스토그램 역투영 결과 영상`을 지정한다.
    - window: 초기에 검색할 영역의 윈도우 위치와 크기를 지정한다. 보통 `입력으로 지정`하여 `업데이트된 결과를 출력으로 다시 반환` 받고,<br>
    그 결과를 `다시 입력으로 지정`하는 방법으로 사용한다.
    - criteria: terminal criteria.
        - e.g., `term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)`
            - 최대 10번 반복하며, 정확도가 1 이하이면(즉, x 또는 y 방향으로의 이동 크기가 1픽셀보다 작으면) 종료한다.


<hr>

### Step #3: Implementation example
<hr>

In [16]:
# 평균 이동 객체 추적 예제
import sys

import cv2


video = cv2.VideoCapture("videos/camshift.avi")

if not video.isOpened():
    print("Video open failed!")
    sys.exit()
else:
    print("Video open succeed!")

# 초기 사각형 영역 설정 - 여기선 사용자 지정값을 사용했지만 ROISelector 또는 객체 검출 기법중에 하나로 위치와 크기 정보를 얻을수도 있다.
x, y, w, h = 135, 220, 100, 100
window = (x, y, w, h)

retval, frame = video.read()

if not retval:
    print("frame read failed!")
    sys.exit()
else:
    print("frame read succeed!")

ROI = frame[y:y+h, x:x+w]
ROI_HSV = cv2.cvtColor(ROI, cv2.COLOR_BGR2HSV)

# HSV 색공간 중 H, S에 대해서만 히스토그램 계산 - 밝기에 대한 변화에 강인하게 하기위해 V 평면은 히스토그램 계산에서 제외한다.
channels = [0, 1]
ranges = [0, 180, 0, 256]
histogram = cv2.calcHist([ROI_HSV], channels, None, [90, 128], ranges)

# Mean Shift 알고리즘 종료 기준 설정
TERM_CRIT = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

# 비디오 매 프레임 처리
while True:
    retval, frame = video.read()
    
    if not retval:
        break
    
    # H, S 히스토그램에 대한 역투영
    frame_HSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    backproject = cv2.calcBackProject([frame_HSV], channels, histogram, ranges, 1)
    
    # Mean Shift
    _, window = cv2.meanShift(backproject, window, TERM_CRIT)
    
    # 추적 결과 출력
    cv2.rectangle(frame, window, (0, 0, 255), 2)
    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(60) == 27:
        print("Video was interrupted!")
        break

video.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

#### NOTE: 추적 대상(귤)을 놓치지 않고 잘 추적하지만 동영상에서 대상이 멀어지게(작게) 되어도 window의 크기가 100 x 100 크기를
#### 유지한 채로 추적한다. 반대로 대상이 가까워져도(크게) window의 크기는 그대로여서 대상을 온전히 표현하지 못 한다는 단점이 있다.
#### 이를 개선하기 위해선 추적 대상의 크기에 따라 window의 크기도 함께 변할 필요가 있다.

Video open succeed!
frame read succeed!


-1

## <font color = "#CC3D3D">Case #3: CamShift Algorithm</font>

<hr>

### Step #1: Concept
<hr>

- 캠시프트(CamShift)
    - 연속 적응 평균 이동(Continuously Adaptive Mean Shift)의 약자이다.
    - 추적하는 객체의 크기가 변하더라도 검색 윈도우의 크기가 고정되어 있는 평균 이동 알고리즘의 단점을 보완했다.

<hr>

- 캠시프트 동작 방법
    - 먼저 `평균 이동 알고리즘`으로 이동 위치를 계산한다.
    - `윈도우의 크기`를 조정한다.
    - 특징 공간을 가장 잘 표현하는 `타원을 검출`한다.
    - `새로운 크기의 윈도우`를 이용하여 다시 평균 이동을 수행한다.
    - <img src="images/markdown/Camshift.png" width="200">

<hr>

- 평균 이동 알고리즘의 단점 개선
    - <img src="images/markdown/.png" width="600">

<hr>

### Step #2: OpenCV function
<hr>

> `캠시프트 추적 함수`

$$\mathsf{{\color{RoyalBlue}cv2.}{\color{Tan}CamShift}(probImage, window, criteria)\rightarrow retval, window}$$
- probImage: probabillity image. 입력 확률 영상
- window: `(x, y, w, h)` 형태의 튜플
- criteria: 알고리즘 종료 기준으로 `(type, maxCount, epsilon)` 형태의 튜플
- retval: `((cx, cy), (width, height), angle)` 형태의 튜플을 반환
- `참고사항:`
    - probImage: 일반적으로 관심 객체에 대한 `히스토그램 역투영 결과 영상`을 지정한다.
    - window: 초기에 검색할 영역의 윈도우 위치와 크기를 지정한다. 보통 `입력으로 지정`하여 `업데이트된 결과를 출력으로 다시 반환` 받고,<br>
    그 결과를 `다시 입력으로 지정`하는 방법으로 사용한다.
    - criteria: terminal criteria.
        - e.g., `term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)`
            - 최대 10번 반복하며, 정확도가 1 이하이면(즉, x 또는 y 방향으로의 이동 크기가 1픽셀보다 작으면) 종료한다.
    - retval: 추적하는 객체의 모양을 나타내는 회전된 사각형 정보를 담고 있다.

<hr>

### Step #3: Implementation example
<hr>

In [33]:
# 캠시프트 알고리즘을 이용한 객체 추적
import sys

import cv2


video = cv2.VideoCapture("videos/camshift.avi")

if not video.isOpened():
    print("Video open failed!")
    sys.exit()
else:
    print("Video open succeed!")
    
# 초기 윈도우 영역 설정 - 여기선 사용자 지정값을 사용했지만 ROISelector 또는 객체 검출 기법중에 하나로 위치와 크기 정보를 얻을수도 있다.
x, y, w, h = 135, 220, 100, 100
window = (x, y, w, h)

retval, frame = video.read()

if not retval:
    print("frame read failed!")
    sys.exit()
else:
    print("frame read succeed!")

ROI = frame[y:y+h, x:x+w]
ROI_HSV = cv2.cvtColor(ROI, cv2.COLOR_BGR2HSV)

# HSV 색공간 중 H, S에 대해서만 히스토그램 계산 - 밝기에 대한 변화에 강인하게 하기위해 V 평면은 히스토그램 계산에서 제외한다.
chennels = [0, 1]
ranges = [0, 180, 0, 256]
histogram = cv2.calcHist([ROI_HSV], channels, None, [90, 128], ranges)

# Mean Shift 알고리즘 종료 기준 설정
TERM_CRIT = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

# 비디오 매 프레임 처리
while True:
    retval, frame = video.read()
    
    if not retval:
        break
    
    # H, S 히스토그램에 대한 역투영
    frame_HSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    backproject = cv2.calcBackProject([frame_HSV], channels, histogram, ranges, 1)
    
    # CamShift
    rotated_rec, window = cv2.CamShift(backproject, window, TERM_CRIT)
    
    # 추적 결과 출력
    cv2.rectangle(frame, window, (0, 0, 255), 2)
    cv2.ellipse(frame, rotated_rec, (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    cv2.imshow("backproject", backproject)
    
    if cv2.waitKey(60) == 27:
        print("Video was interrupted!")
        break
    
video.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

Video open succeed!
frame read succeed!


-1

In [52]:
# 캠시프트 알고리즘을 이용한 객체 추적
import sys

import cv2


video = cv2.VideoCapture("videos/camshift.avi")

if not video.isOpened():
    print("Video open failed!")
    sys.exit()
else:
    print("Video open succeed!")
    
# 초기 윈도우 영역 설정 - 여기선 사용자 지정값을 사용했지만 ROISelector 또는 객체 검출 기법중에 하나로 위치와 크기 정보를 얻을수도 있다.
x, y, w, h = 135, 220, 100, 100
window = (x, y, w, h)

retval, frame = video.read()

if not retval:
    print("frame read failed!")
    sys.exit()
else:
    print("frame read succeed!")

ROI = frame[y:y+h, x:x+w]
ROI_HSV = cv2.cvtColor(ROI, cv2.COLOR_BGR2HSV)

# HSV 색공간 중 H, S에 대해서만 히스토그램 계산 - 밝기에 대한 변화에 강인하게 하기위해 V 평면은 히스토그램 계산에서 제외한다.
chennels = [0, 1]
ranges = [0, 180, 0, 256]
histogram = cv2.calcHist([ROI_HSV], channels, None, [90, 128], ranges)

# Mean Shift 알고리즘 종료 기준 설정
TERM_CRIT = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

# 비디오 매 프레임 처리
while True:
    retval, frame = video.read()
    
    if not retval:
        break
    
    # H, S 히스토그램에 대한 역투영
    frame_HSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    backproject = cv2.calcBackProject([frame_HSV], channels, histogram, ranges, 1)

    threshold = len(backproject[backproject > 128])
    
    # CamShift
    rotated_rec, window = cv2.CamShift(backproject, window, TERM_CRIT)
    
    # 추적 결과 출력
    if threshold < 500:
        cv2.imshow("Frame", frame)
        cv2.imshow("backproject", backproject)
    else:
        cv2.rectangle(frame, window, (0, 0, 255), 2)
        cv2.ellipse(frame, rotated_rec, (0, 255, 0), 2)
        cv2.imshow("Frame", frame)
        cv2.imshow("backproject", backproject)
    
    if cv2.waitKey(60) == 27:
        print("Video was interrupted!")
        break
    
video.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

Video open succeed!
frame read succeed!


-1

## <font color = "#CC3D3D">Case #4: Optical flow</font>

<hr>

### Step #1: Concept
<hr>

- 옵티컬플로우(Optical flow)
    - dfdf

In [ ]:
$$\mathsf{{\color{RoyalBlue} }{\color{Tan} }()\rightarrow }$$
- <img src="images/markdown/.png" width="600">

<hr>

### Step #2: OpenCV function
<hr>

<hr>

### Step #3: Implementation example
<hr>

## <font color = "#CC3D3D">Case #5: OpenCV Tracker</font>

<hr>

### Step #1: Concept
<hr>

- OpenCV 트래커(OpenCV Tracker)
    - dfdf

<hr>

### Step #2: OpenCV function
<hr>

<hr>

### Step #3: Implementation example
<hr>